In [4]:
import re

import numpy as np
import pandas as pd

In [5]:
def read_data(path):    
    header = pd.read_csv(path, nrows=0).columns.tolist()
    EXPECTED_COLS = len(header)
    
    county_idx = header.index("County")
    state_idx = header.index("State")
    
    
    def fix_bad_line(line: list[str]):
        n = len(line)
        
        if n == EXPECTED_COLS + 1:
            merged_county = line[county_idx] + "," + line[state_idx]
            fixed_line = (
                line[:county_idx] +
                [merged_county] +
                line[state_idx + 1:]
            )
            return fixed_line
            
        return line
        
    
    df = pd.read_csv(
        path,
        engine="python",
        encoding="latin-1",
        on_bad_lines=fix_bad_line
    )
    return df
    

def drop_unrecoverable_rows(df):
    df = raw_df.rename(columns={'Unnamed: 0': 'row_id'})
    df['row_id'] = pd.to_numeric(df['row_id'], errors='coerce').astype('Int64')
    df = df[~df.row_id.isna()]
    
    df['ID'] = df['ID'].str.split('-').str[-1]
    df['ID'] = pd.to_numeric(df['ID'], errors='coerce').astype('Int64')
    df = df[~df.ID.isna()]
    return df
    

def fix_shifted_county_records(df):
    shift_idx = df.index[df.County == 'NC']
    cols_after = df.columns[df.columns.get_loc('County') + 1:]
    df.loc[shift_idx, cols_after] = df.loc[shift_idx, cols_after].shift(1, axis=1)
    df.loc[shift_idx, 'County'] = np.nan
    return df
    

def convert_types(df):
    datetime_cols = [
        "Start_Time",
        "End_Time",
        "Weather_Timestamp",
    ]
    
    numeric_cols = [
        "Severity",
        "Start_Lat", "Start_Lng",
        "End_Lat", "End_Lng",
        "Distance(mi)",
        "Temperature(F)",
        "Wind_Chill(F)",
        "Humidity(%)",
        "Pressure(in)",
        "Visibility(mi)",
        "Wind_Speed(mph)",
        "Precipitation(in)"
    ]
    
    categorical_cols = [
        "ID", "Source",
        "Description", "Street", "City", "County", "State",
        "Zipcode", "Country", "Timezone", "Airport_Code",
        "Wind_Direction", "Weather_Condition",
        "Sunrise_Sunset", "Civil_Twilight",
        "Nautical_Twilight", "Astronomical_Twilight"
    ]
    
    bool_cols = [
        "Amenity",
        "Bump",
        "Crossing",
        "Give_Way",
        "Junction",
        "No_Exit",
        "Railway",
        "Roundabout",
        "Station",
        "Stop",
        "Traffic_Calming",
        "Traffic_Signal",
        "Turning_Loop"
    ]
    
    for col in datetime_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format="mixed")
    
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col])
    
    for col in bool_cols:
        if col in df.columns:
            df[col] = df[col].astype(bool).astype(int)
    
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].astype(str)

    return df
    

def unify_column_names(df):
    df.columns = (
        df.columns
          .str.strip()
          .str.lower()
          .str.replace(r"\s+", "_", regex=True)
          .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
    )
    return df

def unify_categorical_variables(df):
    cat_cols = df.select_dtypes(include=["object", "category"]).columns
    
    for col in cat_cols:
        df[col] = (
            df[col]
              .astype(str)
              .str.strip()
              .str.lower()
              .str.replace(r"\s+", "_", regex=True)
        )
    return df
    

In [6]:
path = "../data/raw/US_Accidents_March23.csv"
raw_df = read_data(path)
df = drop_unrecoverable_rows(raw_df)
df = fix_shifted_county_records(df)
df = convert_types(df)
# df = unify_column_names(df)
# df = unify_categorical_variables(df)

In [16]:
    df['min_temperature_f'] = (
        df['Temperature_Range(F)']
        .str.split('-').str[0]
        .astype(float, errors='ignore')
    )
    df['max_temperature_f'] = (
        df['Temperature_Range(F)']
        .str.split('-').str[1]
        .astype(float, errors='ignore')
    )
    df = df.drop(columns=['Temperature_Range(F)'], axis=1)

In [17]:
df

,row_id,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,min_temperature_f,max_temperature_f
0,0,3650461,Source1,2,2023-03-31 17:09:16,2023-03-31 18:09:49,45.676472,-94.174568,45.666976,-94.176184,...,0,0,0,1,Day,Day,Day,Day,31.0,35.0
1,1,3650462,Source1,2,2023-02-27 20:55:00,2023-02-27 23:36:34,41.391812,-75.467365,41.398370,-75.484203,...,0,0,0,1,Night,Night,Night,Night,30.0,34.0
2,2,3650463,Source1,2,2023-02-26 09:31:45,2023-02-26 11:58:19,40.850278,-73.946140,40.846945,-73.933651,...,0,0,0,1,Day,Day,Day,Day,29.0,33.0
3,3,3650464,Source1,2,2023-03-21 17:46:42,2023-03-21 18:31:50,33.714992,-84.300188,33.714532,-84.266758,...,0,0,0,1,Day,Day,Day,Day,59.0,63.0
4,4,3650465,Source1,2,2023-03-24 04:46:30,2023-03-24 09:22:00,33.927671,-118.266385,33.929608,-118.343065,...,0,0,0,1,Night,Night,Night,Night,55.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246660,246628,5464675,Source1,2,2023-01-19 14:52:14,2023-01-19 16:07:14,29.697432,-95.423674,29.697499,-95.425230,...,1,0,0,1,Day,Day,Day,Day,70.0,74.0
246661,246629,5464680,Source1,2,2023-02-20 15:20:00,2023-02-20 15:52:00,33.869027,-84.366612,33.853174,-84.370037,...,0,0,0,1,Day,Day,Day,Day,72.0,76.0
246662,246630,5464682,Source1,2,2023-01-25 04:53:15,2023-01-25 06:11:02,39.603673,-86.069712,39.609702,-86.072118,...,0,0,0,1,Night,Night,Night,Night,31.0,35.0
246663,246631,5464712,Source1,2,2023-01-24 10:23:26,2023-01-24 11:38:26,32.917276,-96.717765,32.916783,-96.717758,...,0,0,0,1,Day,Day,Day,Day,41.0,45.0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246633 entries, 0 to 246664
Data columns (total 48 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   row_id                 246633 non-null  Int64         
 1   ID                     246633 non-null  object        
 2   Source                 246633 non-null  object        
 3   Severity               246633 non-null  int64         
 4   Start_Time             246633 non-null  datetime64[ns]
 5   End_Time               246633 non-null  datetime64[ns]
 6   Start_Lat              246633 non-null  float64       
 7   Start_Lng              246633 non-null  float64       
 8   End_Lat                246633 non-null  float64       
 9   End_Lng                246633 non-null  float64       
 10  Distance(mi)           246633 non-null  float64       
 11  Description            246633 non-null  object        
 12  Street                 246633 non-null  object   

In [12]:
df['min_temperature_f'] = (
    df['Temperature_Range(F)'].str.split('-').str[0].str.strip().astype(float, errors='ignore')
)
# df['max_temperature_f'] = (
    # df['Temperature_Range(F)'].str.split('-').str[1].str.strip().astype(float)
# )
# df = df.drop(columns=['Temperature_Range(F)'], axis=1)